## Score en Kaggle: 0.85193

In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
import xgboost as xgb

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
import category_encoders

In [4]:
from sklearn.model_selection import RandomizedSearchCV

In [5]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [6]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [8]:
processed = pd.read_csv('features_04.csv')

In [9]:
processed = processed.set_index(processed.columns[0])
processed.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,primer_visita,ultima_visita,cant_eventos,permanencia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,...,1362,False,False,True,False,False,True,False,True,False
00091926,viewed product,iPhone 6S,Computer,0,34,372,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,...,1155,False,True,True,True,True,True,True,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,...,12924,True,True,True,True,False,True,False,True,False


In [10]:
predict_features = processed.loc[processed.index.isin(predict['person'])].drop(['primer_visita','ultima_visita'],axis=1)

In [11]:
predict_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
00091926,viewed product,iPhone 6S,Computer,0,34,372,448,28,0,16.000000,...,1155,False,True,True,False,False,False,True,True,False
00091a7a,brand listing,iPhone 6S,Smartphone,0,1,3,10,1,66,10.000000,...,12924,False,True,False,False,False,False,True,True,False
000ba417,viewed product,Samsung Galaxy A3 Duos,Computer,1,6,153,206,10,5,20.600000,...,1155,False,True,True,True,True,True,True,True,False
000e4d9e,viewed product,Samsung Galaxy S6 Flat,Computer,0,13,339,411,15,1,27.400000,...,335,False,True,True,True,False,True,True,True,False
000e619d,viewed product,Motorola Moto G2 3G Dual,Computer,0,5,28,68,3,13,22.666667,...,1155,True,True,True,True,False,True,True,True,False


In [12]:
training_features = processed.loc[processed.index.isin(a_predecir['person'])]
training_features['label'] = a_predecir.set_index('person')['label']


training_labels = training_features['label']
training_features = training_features.drop(['primer_visita','ultima_visita','label'],axis=1)

C:\Users\javif\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
training_features.head()

,mayor_evento,prod_visto,dispos,compras,visitas,cant_prod_vistos,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,...,personas_mismo_scrnres,busco_productos,clickeo_ad,hizo_checkout,entro_desde_motor,hizo_conversion,filtro_busquedas,busco_marca,visito_sitio,hizo_lead
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,checkout,Ninguno,Computer,0,2,0,6,1,14,6.000000,...,1362,False,False,True,False,False,True,False,True,False
000c79fe,searched products,iPhone 7,Smartphone,0,1,3,17,1,2,17.000000,...,12924,True,True,True,True,False,True,False,True,False
001802e4,ad campaign hit,iPhone 6S,Smartphone,0,1,4,19,1,0,19.000000,...,76,True,True,True,False,False,True,False,True,False
0019e639,viewed product,Motorola Moto G2 3G Dual,Computer,2,19,189,471,115,9,4.095652,...,8873,True,True,True,True,True,True,True,True,False
001b0bf9,viewed product,Samsung Galaxy S8 Plus,Ninguno,1,0,2,7,1,9,7.000000,...,588,False,False,True,False,True,True,True,False,False


## Mean encoding

In [14]:
tg = category_encoders.TargetEncoder(smoothing=True)
encoded_training = tg.fit_transform(np.array(training_features),np.array(training_labels))
encoded_predict = tg.transform(np.array(predict_features))

In [15]:
encoded_training.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,0.002778,0.042654,0.068258,0,2,0,6,1,14,6.000000,...,1362,False,False,True,False,False,True,False,True,False
1,0.055313,0.050139,0.036301,0,1,3,17,1,2,17.000000,...,12924,True,True,True,True,False,True,False,True,False
2,0.031056,0.033590,0.036301,0,1,4,19,1,0,19.000000,...,76,True,True,True,False,False,True,False,True,False
3,0.049121,0.030380,0.068258,2,19,189,471,115,9,4.095652,...,8873,True,True,True,True,True,True,True,True,False
4,0.049121,0.060150,0.017361,1,0,2,7,1,9,7.000000,...,588,False,False,True,False,True,True,True,False,False


In [16]:
params={
    'n_estimators':stats.randint(150,500),
    'learning_rate':stats.uniform(0.01,0.3),
    'subsample':stats.uniform(0.3,0.7),
    'min_child_weight':[1,5,10],
    'max_depth':[3,10,6],
    'gamma':stats.randint(0,10),
    'colsample_bytree':stats.uniform(0.4,0.6)
}
grid = RandomizedSearchCV(xgb.XGBClassifier(n_jobs=-1),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=1,
                          n_iter=100)

In [18]:
xtr,xte,ytr,yte=train_test_split(encoded_training,training_labels)

In [19]:
grid.fit(xtr,ytr)

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed:  7.9min finished


RandomizedSearchCV(cv=2, error_score='raise',
          estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=100, n_jobs=1,
          param_distributions={'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020F28A2F9E8>, 'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020F28A2F668>, 'subsample': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020F28A2FF28>, 'min_...D8>, 'colsample_bytree': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000020F27C7A048>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
    

In [20]:
grid.score(X=xte,y=yte)

0.87203633775618

In [21]:
grid.best_params_

{'colsample_bytree': 0.606653667095343,
 'gamma': 7,
 'learning_rate': 0.016238752458245277,
 'max_depth': 10,
 'min_child_weight': 5,
 'n_estimators': 346,
 'subsample': 0.7847190225361189}

## Prediccion

In [22]:
predictor = grid.best_estimator_

In [23]:
predictor.feature_importances_

array([0.01234568, 0.09635652, 0.00602228, 0.04426378, 0.04245709,
       0.05389943, 0.0809997 , 0.05510388, 0.13249022, 0.05811502,
       0.01144234, 0.        , 0.04125264, 0.        , 0.03794038,
       0.04516712, 0.07618187, 0.00572117, 0.00331226, 0.01866908,
       0.01806685, 0.00873231, 0.00060223, 0.01355014, 0.02318579,
       0.01716351, 0.00120446, 0.06323396, 0.01204456, 0.01445348,
       0.00120446, 0.0042156 , 0.        , 0.00060223], dtype=float32)

In [24]:
predictor.fit(encoded_training,training_labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.606653667095343, gamma=7,
       learning_rate=0.016238752458245277, max_delta_step=0, max_depth=10,
       min_child_weight=5, missing=None, n_estimators=346, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.7847190225361189)

In [26]:
prediction_submit = predictor.predict_proba(encoded_predict)[:,1]

In [27]:
predict_features['label']=prediction_submit

In [29]:
submit = predict.set_index('person')

In [30]:
submit['label'] = predict_features['label']

In [31]:
submit.to_csv('submit.csv')